# Get started with the Estimator primitive

Learn how to set up and use the Estimator primitive program.


## Overview

The Estimator primitive lets you efficiently calculate and interpret expectation values of quantum operators required for many algorithms. You can specify a list of circuits and observables, then evaluate expectation values and variances for a given parameter input.  


## Prepare the environment

1. Follow the steps in the [getting started guide](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/getting_started.html) to get your quantum service instance ready to use.

2. You'll need at least one circuit to submit to the program. Our examples all have circuits in them, but if you want to submit your own circuit, you can use Qiskit to create one. To learn how to create circuits by using Qiskit, see the [Circuit basics tutorial](https://qiskit.org/documentation/tutorials/circuits/01_circuit_basics.html).

3. Create a list of observables. Observables let you define the properties of the circuit that are relevant to your problem and enable you to efficiently measure their expectation value. For simplicity, you can use the [PauliSumOp class](https://qiskit.org/documentation/stubs/qiskit.opflow.primitive_ops.html#module-qiskit.opflow.primitive_ops) in Qiskit to define them, as illustrated in the example below.


## Start a session

With Qiskit Runtime primitives, we introduce the concept of a session or a factory that allows you to define a job as a collection of iterative calls to the quantum computer. When you start a session, it caches the data you send so it doesn't have to be transmitted to the Quantum Datacenter on each iteration.

### Create an Estimator instance

The Estimator class takes in a runtime options dictionary to control the execution environment.

- **options**: Runtime options dictionary that control the execution environment.
    * **backend**: Optional instance of :class:`qiskit_ibm_runtime.IBMBackend` class or
        string name of backend, if not specified a backend will be selected
        automatically (IBM Cloud only).
    * **image**: the runtime image used to execute the program, specified in
        the form of ``image_name:tag``. Not all accounts are
        authorized to select a different image.
    * **log_level**: logging level to set in the execution environment. The valid
        log levels are: ``DEBUG``, ``INFO``, ``WARNING``, ``ERROR``, and ``CRITICAL``.
        The default level is ``WARNING``.

The Estimator class also takes in the following arguments as settings:

- **transpilation_settings**: (EXPERIMENTAL setting, can break between releases without warning)
    Qiskit transpiler settings. The transpilation process converts
    operations in the circuit to those supported by the backend, swaps qubits with the
    circuit to overcome limited qubit connectivity and some optimizations to reduce the
    circuit's gate count where it can.

    * **skip_transpilation**: Transpilation is skipped if set to True.
        False by default.

    * **optimization_settings**:
        * **level**: How much optimization to perform on the circuits.
            Higher levels generate more optimized circuits,
            at the expense of longer transpilation times.
            * 0: no optimization
            * 1: light optimization
            * 2: heavy optimization
            * 3: even heavier optimization
            If ``None``, level 1 will be chosen as default.

- **resilience_settings**: (EXPERIMENTAL setting, can break between releases without warning)
    Using these settings allows you to build resilient algorithms by
    leveraging the state of the art error suppression, mitigation and correction techniques.

    * **level**: How much resilience to build against errors.
        Higher levels generate more accurate results,
        at the expense of longer processing times.
        * 0: no resilience
        * 1: light resilience
        If ``None``, level 0 will be chosen as default.


With these arguments, you can create an Estimator instance with the desired settings. Example:

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp

service = QiskitRuntimeService()

options = { "backend": "ibmq_qasm_simulator" }
transpilation_settings = { "optimization_level": 1 }

psi1 = RealAmplitudes(num_qubits=2, reps=2)
psi2 = RealAmplitudes(num_qubits=2, reps=3)

H1 = SparsePauliOp.from_list([("II", 1), ("IZ", 2), ("XI", 3)])
H2 = SparsePauliOp.from_list([("IZ", 1)])
H3 = SparsePauliOp.from_list([("ZI", 1), ("ZZ", 1)])

with Session(service) as session:
    estimator = session.estimator(options=options, transpilation_settings=transpilation_settings)

## Write to & read from a session

Running a job and returning the results are done by writing to and reading from the session. After the results are returned, the session is automatically closed.

### Specify program inputs

The Estimator.run() method takes in the following arguments:

- **circuits**: A list of (parameterized) circuits that you want to investigate.
- **observables**: A list of observables to measure the expectation values.
- **parameters**: A list of parameters for the parameterized circuits. It should be omitted if the circuits provided are not parameterized.
- **parameter_values**: an optional list of concrete parameters to be bound.
- **run_options**: A collection of kwargs passed to `backend.run()`.
    - **shots**: Number of repetitions of each circuit, for sampling.
    - **qubit_lo_freq**: List of default qubit LO frequencies in Hz.
    - **meas_lo_freq**: List of default measurement LO frequencies in Hz.
    - **schedule_los**: Experiment LO configurations, frequencies are given in Hz.
    - **rep_delay**: Delay between programs in seconds. Only supported on certain
        backends (if ``backend.configuration().dynamic_reprate_enabled=True``).
    - **init_qubits**: Whether to reset the qubits to the ground state for each shot.
    - **use_measure_esp**: Whether to use excited state promoted (ESP) readout for measurements
        which are the terminal instruction to a qubit. ESP readout can offer higher fidelity
        than standard measurement sequences.

You can find more details in [the Estimator API reference](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.Estimator.html).

### Run the job & print results

Run the job, specifying your previously defined inputs and options.  Use `circuits`, `observables`, and `parameter_values` to use a specific parameter and observable with the specified circuit.

For example, this line `psi1_H23_result = estimator(circuits=[0, 0], observables=[1, 2], parameter_values=[theta1]*2)` specifies the following:

- Return the value for using observable `H2` and parameter `theta1` with the circuit `psi1`.
- Return the value for using observable `H3` and parameter `theta1` with the circuit `psi1`.

You can also pass circuits and observables as objects.
For example, `psi1_H23_result = estimator(circuits=[ps1, ps1], observables=[H2, H3], parameter_values=[theta1]*2)`

In [ ]:
with Session(service) as session:
    estimator = session.estimator()
    estimator.options.backend = 'ibmq_qasm_simulator'
    estimator.settings.transpilation.optimization_level = 1

    theta1 = [0, 1, 1, 2, 3, 5]
    theta2 = [0, 1, 1, 2, 3, 5, 8, 13]
    theta3 = [1, 2, 3, 4, 5, 6]

    # calculate [ <psi1(theta1)|H1|psi1(theta1)> ]
    psi1_H1 = estimator.run(circuits=[psi1], observables=[H1], parameter_values=[theta1])
    print(psi1_H1.result())

    # calculate [ <psi1(theta1)|H2|psi1(theta1)>, <psi1(theta1)|H3|psi1(theta1)> ]
    psi1_H23 = estimator.run(circuits=[psi1, psi1], observables=[H2, H3], parameter_values=[theta1]*2)
    print(psi1_H23.result())

    # calculate [ <psi2(theta2)|H2|psi2(theta2)> ]
    psi2_H2 = estimator.run(circuits=[psi2], observables=[H2], parameter_values=[theta2])
    print(psi2_H2.result())

    # calculate [ <psi1(theta1)|H1|psi1(theta1)>, <psi1(theta3)|H1|psi1(theta3)> ]
    psi1_H1_job = estimator.run(circuits=[psi1, psi1], observables=[H1, H1], parameter_values=[theta1, theta3])
    print(psi1_H1_job.result())

    # calculate [ <psi1(theta1)|H1|psi1(theta1)>,
    #             <psi2(theta2)|H2|psi2(theta2)>,
    #             <psi1(theta3)|H3|psi1(theta3)> ]
    psi12_H23 = estimator.run(circuits=[psi1, psi2, psi1], observables=[H1, H2, H3], parameter_values=[theta1, theta2, theta3])
    print(psi12_H23.result())

EstimatorResult(values=array([1.59570312]), metadata=[{'variance': 8.661632537841797, 'shots': 1024}])
EstimatorResult(values=array([-0.5390625 ,  0.07421875]), metadata=[{'variance': 0.70941162109375, 'shots': 1024}, {'variance': 1.9894332885742188, 'shots': 1024}])
EstimatorResult(values=array([0.17773438]), metadata=[{'variance': 0.9684104919433594, 'shots': 1024}])


The results align with the parameter - circuit - observable tuples specified previously.  For example, the first result: `EstimatorResult(values=array([1.53710938]), metadata=[{'variance': 9.249675750732422, 'shots': 1024}])` is the output of the parameter labeled `theta1` and observable `H1` being sent to the first circuit.